In [ ]:
#tensorflow stock predictor
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import scipy

# Define the ticker symbol and download the historical data
ticker = "AAPL"
data = yf.download(ticker, start="2023-01-01", end="2023-03-31")



# Define the input and output variables
X = data[["Open", "High", "Low", "Close", "Volume"]]
y = data[["Adj Close"]]

# Scale the data using a MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X_scaled[0:train_size,:], X_scaled[train_size:len(X),:]
y_train, y_test = y_scaled[0:train_size,:], y_scaled[train_size:len(y),:]

# Define the neural network model
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_dim=5, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Train Loss: {train_loss:.5f}')
print(f'Test Loss: {test_loss:.5f}')

# Plot the training and validation loss over time
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Make predictions using the trained model
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# Plot the predicted and actual prices over time
plt.plot(predictions)
plt.plot(y_test)
plt.title('Predicted vs Actual Stock Price')
plt.ylabel('Price')
plt.xlabel('Time')
plt.legend(['Predicted', 'Actual'], loc='upper left')
plt.show()
